In [32]:
# This is used to help test our files with SCIKA

In [12]:
from sklearn import metrics, svm
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing

from pandas import DataFrame, read_csv
import pandas as pd
import os
import csv
import io
import scipy

from sklearn.feature_selection import VarianceThreshold 
from sklearn.feature_selection import SelectKBest, chi2, f_classif



In [133]:
# Columns to drop from the data frame so that the analyzers do not use them
path = r"C:\Users\sam\Documents\000 KSU Final\CIS 590\StockData\AppleData\10W_Win_4W_Wait_Apple_Price_Change.csv"

classifier_file_path = r"C:\Users\sam\Documents\000 KSU Final\CIS 590\StockData\ClassifierDataStuff\AppleDummy.csv"

columns_to_drop = ['Index', 'Total Vol', '100D MA', '1W Price Change']

# Columns to either test or use as truth values
truth_column_name = '1W Price Increase'
test_column_list = ['Avg Vol', 'Min Vol', 'Max Vol', 
                    'Min', 'Max', 'Window MA', 
                    'RSI 2W', 'RSI Window', 'Price Change', 
                    'Price Increase', 'Avg Price Change', 'RSI Oversold', 'RSI Undersold' ]


df = pd.read_csv(path)
for column_name in columns_to_drop:
    df = df.drop([column_name], axis = 1)
    

# Returns pearson correlation and p-value
for column in test_column_list:
    pearson_val = scipy.stats.pearsonr(df[column], df[truth_column_name])
    print("Pearson value of ",  column, " is " , pearson_val)





Pearson value of  Avg Vol  is  (0.10386590176975419, 0.0002227263104327574)
Pearson value of  Min Vol  is  (0.04812173113889054, 0.0878654031662501)
Pearson value of  Max Vol  is  (0.0447155219482685, 0.11277769210256339)
Pearson value of  Min  is  (-0.05592057219483085, 0.047280701685099916)
Pearson value of  Max  is  (-0.02776451211492496, 0.3249373340182726)
Pearson value of  Window MA  is  (-0.036386588434498005, 0.1969715295236932)
Pearson value of  RSI 2W  is  (-0.04366953123269225, 0.1214540882521701)
Pearson value of  RSI Window  is  (-0.15081265460708645, 7.58812353952941e-08)
Pearson value of  Price Change  is  (-0.015271973783423152, 0.5882460736631334)
Pearson value of  Price Increase  is  (0.02508023837793797, 0.3739156404091461)
Pearson value of  Avg Price Change  is  (-0.1520718728701283, 5.8910333966784255e-08)
Pearson value of  RSI Oversold  is  (-0.007292540430143432, 0.7960185978682958)
Pearson value of  RSI Undersold  is  (0.07530286184174387, 0.007516114010164867)


In [134]:
# Shuffles the values before allocation to provide evenly distributed data
df = shuffle(df)
df

,Avg Vol,Min Vol,Max Vol,Min,Max,Window MA,RSI 2W,RSI Window,Price Change,Price Increase,1W Price Increase,Avg Price Change,RSI Oversold,RSI Undersold
1052,199315828.6,80113600,426510000,53.152500,81.962502,71.046143,54.398056,47.786363,1.022499,True,1,-0.093107,False,False
1237,109783064.3,46691300,192541500,107.320000,145.089996,123.753714,74.382764,57.856775,-1.100006,False,0,0.305000,True,False
953,101026674.3,46617600,221652400,51.055000,67.750000,60.281929,52.681856,67.664110,0.389999,True,1,0.213357,False,False
411,107346737.1,62468000,286856000,35.724998,41.235001,38.925250,75.959692,57.758441,0.247501,True,1,0.050929,True,False
936,105688142.9,63755200,221652400,49.787498,66.220001,56.704500,74.564090,67.729709,-0.457497,False,1,0.227143,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,155798880.0,77645600,458408400,22.367500,28.097500,25.017500,38.418157,42.590617,0.299999,True,1,-0.039714,False,False
261,113362982.9,56998000,447940000,27.062500,35.070000,31.060536,65.398176,78.156725,0.014999,True,1,0.099071,False,False
1149,172547422.9,89001600,374336800,83.144997,137.979996,105.049249,31.746653,58.973877,3.240005,True,1,0.372929,False,False
105,159056331.4,83392400,458408400,22.367500,27.022499,24.291714,74.662686,48.191409,0.347500,True,1,-0.009179,True,False


In [135]:
# Creates the testing and training samples for Scikit to use
# Allocates one third of values as a training set and the rest as a testing set
train = df.iloc[lambda x: x.index % 4 == 0]
test = df.iloc[lambda x: x.index % 4 != 0]
print('test length: ',len(test))
print('train length: ',len(train))

#test=df.iloc[:300,:]
#train=df.iloc[300:,:]

# Sets the training value to Price Increase (bool) and removes it from the data so it doesn't use itself
train_labels=train[truth_column_name].tolist()
train=train.drop([truth_column_name],axis=1)
train_samples=train.values.tolist()

test_labels=test[truth_column_name].tolist()
test=test.drop([truth_column_name],axis=1)
test_samples=test.values.tolist()

test length:  944
train length:  315


In [136]:
# Modifies the data into a format that Scikit can use (turns boolean values into 0 for False, 1 for True)
for i in range (len(train_labels)):
  if train_labels[i]==False:
    train_labels[i]=0
  if train_labels[i]==True:
    train_labels[i]=1

for i in range (len(test_labels)):
  if test_labels[i]==False:
    test_labels[i]=0
  if test_labels[i]==True:
    test_labels[i]=1


In [137]:
# List of classifiers to apply to our data
classifiers = [
#KNeighborsClassifier(3),
#SVC(kernel="rbf", C=0.025, probability=True),
#NuSVC(probability=True),
#DecisionTreeClassifier(),
#RandomForestClassifier()
#AdaBoostClassifier(),
#GradientBoostingClassifier(),
#GaussianNB(),
#LinearDiscriminantAnalysis(),
#QuadraticDiscriminantAnalysis()
    
DummyClassifier()
]

In [138]:
# list and running total for accuracy of classifiers

acc_total = 0
acc_list =[]
clf_name_list =[]
df_temp = pd.DataFrame()
clf_name=""



lab_enc = preprocessing.LabelEncoder()
train_labels_encoded = lab_enc.fit_transform(train_labels)
test_labels_encoded = lab_enc.fit_transform(test_labels)

# Runs all the classifiers from the previous list and determines which has the highest accuracy
for clf in classifiers:
    clf.fit(train_samples, train_labels_encoded)
    res=clf.predict(test_samples)
    #print("Prediction: ",res)
    acc = accuracy_score(test_labels_encoded, res)
    acc_string = str(acc)
    acc_list.append(acc)
    acc_total = acc_total + acc
    
    print (clf.__class__.__name__+" Accuracy: "+str(acc))
    clf_name_list.append(clf.__class__.__name__)
    clf_name = clf.__class__.__name__
path_list = path.split(os.sep)
path_end = path_list[8]
path_list = [path_end] * len(acc_list)

temp_row = ["1",clf_name, acc_string, path_end]
temp_string_row = ','.join(temp_row)
print(temp_row)

# Creates the temporary DataFrame which contains classifier results
df_temp['Index'] = [1]
df_temp['Classifier']= clf_name_list
df_temp['Accuracy']= acc_list
df_temp['FilePath']= path_list

try: 
    with open(classifier_file_path, 'a') as fd:
        writer = csv.writer(fd)
        writer.writerow(temp_row)
except FileNotFoundError:
    df_temp.to_csv(classifier_file_path)

print("\n Current Results: \n", df_temp)



DummyClassifier Accuracy: 0.5360169491525424
['1', 'DummyClassifier', '0.5360169491525424', '10W_Win_4W_Wait_Apple_Price_Change.csv']

 Current Results: 
    Index       Classifier  Accuracy                                FilePath
0      1  DummyClassifier  0.536017  10W_Win_4W_Wait_Apple_Price_Change.csv


C:\Users\sam\anaconda3\lib\site-packages\sklearn\dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


In [139]:
# Removes duplicate data from using the pre-existing results file.
df_duplicates = pd.read_csv(classifier_file_path)
df_duplicates.drop_duplicates(subset = "FilePath", inplace = True)
df_duplicates= df_duplicates.drop(['Index'], axis = 1)
df_duplicates = df_duplicates.drop([3])
df_duplicates

,Classifier,Accuracy,FilePath
0,DummyClassifier,0.584106,10W_Win_3W_Wait_Apple_Price_Change.csv
1,DummyClassifier,0.584106,10W_Win_2W_Wait_Apple_Price_Change.csv
2,DummyClassifier,0.556291,10W_Win_1W_Wait_Apple_Price_Change.csv
4,DummyClassifier,0.509934,3W_Win_1W_Wait_Apple_Price_Change.csv
5,DummyClassifier,0.517881,4W_Win_1W_Wait_Apple_Price_Change.csv
6,DummyClassifier,0.515232,5W_Win_1W_Wait_Apple_Price_Change.csv
7,DummyClassifier,0.531126,6W_Win_1W_Wait_Apple_Price_Change.csv
8,DummyClassifier,0.528477,7W_Win_1W_Wait_Apple_Price_Change.csv
9,DummyClassifier,0.543046,8W_Win_1W_Wait_Apple_Price_Change.csv
10,DummyClassifier,0.517881,9W_Win_1W_Wait_Apple_Price_Change.csv


In [104]:
# Code to test our predictions with Scikit
correct_samples=0

average_accuracy = acc_total / len(classifiers)
print(len(classifiers), "different classifiers used for analysis")
print("Average accuracy: ", average_accuracy)
max = 0
min = 1

for value in acc_list:
    if value < min:
        min = value
    if value > max:
        max = value
        
print("Worst Accuracy: ", min)
print("Best Accuracy: ", max)

1 different classifiers used for analysis
Average accuracy:  0.5688559322033898
Worst Accuracy:  0.5688559322033898
Best Accuracy:  0.5688559322033898


In [19]:
# Uses a variance threshold to select the best samples
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(train_samples) 


X_new = SelectKBest(f_classif, k=2).fit_transform(train_samples, train_labels)

In [20]:
X_new

array([[43.67066533, 71.58873605],
       [77.20743253, 66.95249567],
       [61.26104413, 61.35420785],
       ...,
       [51.80853217, 35.48857241],
       [86.8661219 , 73.08643386],
       [57.44851947, 53.83734565]])

In [ ]:
T_of_WindowMA_and_PriceChange = '''
The two-tailed P value is less than 0.0001
Group     Window MA          Price Change
Mean      67.78465945332     0.10198676507
SD        27.75433845588     1.74283004779
SEM       1.01008379636      0.06342807968
N         755
'''


In [24]:
# Creates the DataFrame which holds the CSV file containing our  average window data
df_over_window = pd.read_csv(r"C:\Users\sam\Documents\000 KSU Final\CIS 590\StockData\Temp_Group_Window_Market_Data_2W.csv")

for column_name in columns_to_drop:
    df_over_window = df_over_window.drop([column_name], axis = 1)
    
df_over_window=shuffle(df_over_window)
df_over_window

,Unnamed: 0,Avg Vol,Min Vol,Max Vol,Min,Max,Window MA,RSI 2W,RSI Window,2W Price Increase
2245,2245,1.966486e+07,3265500,53225700,48.110001,54.930000,51.105334,28.237751,33.583398,True
1511,1511,1.463853e+07,8277200,25184300,42.189999,48.619999,45.383667,52.902294,36.890954,False
4839,4839,5.256517e+07,29177600,105168400,28.010000,31.170000,29.623000,26.810265,47.416023,False
1215,1215,6.216427e+06,2019300,11207400,281.140015,338.000000,309.653333,71.891012,65.470744,True
4660,4660,4.898707e+07,33062900,69070700,30.160000,31.910000,31.074667,57.167209,53.170728,False
...,...,...,...,...,...,...,...,...,...,...
96,96,6.661365e+07,32094000,113048600,14.310000,17.340000,15.884333,64.536888,58.938054,True
1573,1573,1.201424e+07,6331900,20774700,41.450001,44.970001,42.588000,69.976077,40.894909,True
403,403,5.041109e+07,34446700,83681100,27.430000,32.049999,30.107333,32.743709,39.444854,True
1137,1137,6.469880e+06,3682800,10847500,282.790009,336.399994,306.224664,47.756994,41.713094,False


In [26]:
# Creates the testing and training samples for Scikit to use
# Allocates one third of values as a training set and the rest as a testing set
window_train = df_over_window.iloc[lambda x: x.index % 3 == 0]
window_test = df_over_window.iloc[lambda x: x.index % 3 != 0]
print('test length: ',len(window_test))
print('train length: ',len(window_train))

#test=df.iloc[:300,:]
#train=df.iloc[300:,:]

# Sets the training value to Price Increase (bool) and removes it from the data so it doesn't use itself
window_train_labels=window_train['2W Price Increase'].tolist()
window_train=window_train.drop(['2W Price Increase'],axis=1)
window_train_samples=window_train.values.tolist()

window_test_labels=window_test['2W Price Increase'].tolist()
window_test=window_test.drop(['2W Price Increase'],axis=1)
window_test_samples=window_test.values.tolist()

test length:  3523
train length:  1762


In [27]:
# Modifies the data into a format that Scikit can use (turns boolean values into 0 for False, 1 for True)
for i in range (len(window_train_labels)):
  if window_train_labels[i]==False:
    window_train_labels[i]=0
  if window_train_labels[i]==True:
    window_train_labels[i]=1

for i in range (len(window_test_labels)):
  if window_test_labels[i]==False:
    window_test_labels[i]=0
  if window_test_labels[i]==True:
    window_test_labels[i]=1


In [36]:
# list and running total for accuracy of classifiers
window_acc_total = 0
window_acc_list =[]

# Runs all the classifiers from the previous list and determines which has the highest accuracy
for clf in classifiers:
    clf.fit(window_train_samples, window_train_labels)
    window_res=clf.predict(window_test_samples)
    acc = accuracy_score(window_test_labels, window_res)
    print (clf.__class__.__name__+" Accuracy: "+str(acc))
    acc = accuracy_score(window_test_labels, window_res)
    window_acc_list.append(acc)
    window_acc_total = acc_total + acc


KNeighborsClassifier Accuracy: 0.7933579335793358
DecisionTreeClassifier Accuracy: 0.8189043428895827
RandomForestClassifier Accuracy: 0.8705648594947488
AdaBoostClassifier Accuracy: 0.6440533636105592
GradientBoostingClassifier Accuracy: 0.7553221686063014
GaussianNB Accuracy: 0.5611694578484246
LinearDiscriminantAnalysis Accuracy: 0.5852966221969912
QuadraticDiscriminantAnalysis Accuracy: 0.5852966221969912
DummyClassifier Accuracy: 0.5095089412432586


C:\Users\sam\anaconda3\lib\site-packages\sklearn\dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


In [37]:
# Code to test our predictions with Scikit
correct_samples=0

average_accuracy = window_acc_total / len(classifiers)
print(len(classifiers), "different classifiers used for analysis")
print("Average accuracy: ", average_accuracy)
max = 0
min = 1

for value in window_acc_list:
    print(value)
    if value < min:
        min = value
    if value > max:
        max = value
        
print("Worst Accuracy: ", min)
print("Best Accuracy: ", max)

9 different classifiers used for analysis
Average accuracy:  0.5686756867568676
0.7933579335793358
0.8189043428895827
0.8705648594947488
0.6440533636105592
0.7553221686063014
0.5611694578484246
0.5852966221969912
0.5852966221969912
0.5095089412432586
Worst Accuracy:  0.5095089412432586
Best Accuracy:  0.8705648594947488


In [ ]:
two_week_pearson = '''
Pearson value of  Avg Vol  is  (-0.06816651166363226, 0.03054306598846368)
Pearson value of  Min Vol  is  (-0.0695175961899923, 0.02738754132938469)
Pearson value of  Max Vol  is  (-0.08957856522502271, 0.004443786400384686)
Pearson value of  Min  is  (-0.06222844010210368, 0.04836214879541458)
Pearson value of  Max  is  (-0.06779427528395819, 0.03146540320121367)
Pearson value of  Window MA  is  (-0.06430632579391746, 0.04132772126354686)
Pearson value of  RSI 2W  is  (0.05137524170100003, 0.1032378350460256)
Pearson value of  RSI Window  is  (0.05134561976807255, 0.10343701221966668)
Pearson value of  Price Change  is  (0.0016287211177530639, 0.9588311028457459)
Pearson value of  Price Increase  is  (0.02144803761963549, 0.496600020701046)
Pearson value of  Avg Price Change  is  (0.056201208768004454, 0.0746446397478281)
Pearson value of  RSI Oversold  is  (0.03516679552330661, 0.26488642020644365)
Pearson value of  RSI Undersold  is  (0.02386835513775085, 0.4492968228625301)
'''
three_week_pearson = '''
Pearson value of  Avg Vol  is  (-0.04823845496600528, 0.12607789265870958)
Pearson value of  Min Vol  is  (-0.0371058124305515, 0.239422615198146)
Pearson value of  Max Vol  is  (-0.04962151749722914, 0.11556459025512342)
Pearson value of  Min  is  (-0.07036760899213987, 0.025549677024658053)
Pearson value of  Max  is  (-0.06723862856082727, 0.032886537624642664)
Pearson value of  Window MA  is  (-0.06783437128530379, 0.031364919113161374)
Pearson value of  RSI 2W  is  (0.05137524170100003, 0.1032378350460256)
Pearson value of  RSI Window  is  (0.06691493931356744, 0.03373943817820531)
Pearson value of  Price Change  is  (0.0016287211177530639, 0.9588311028457459)
Pearson value of  Price Increase  is  (0.02144803761963549, 0.496600020701046)
Pearson value of  Avg Price Change  is  (0.045156040838969416, 0.1521726676055877)
Pearson value of  RSI Oversold  is  (0.03516679552330661, 0.26488642020644365)
Pearson value of  RSI Undersold  is  (0.02386835513775085, 0.4492968228625301)
'''
four_week_pearson = '''
Pearson value of  Avg Vol  is  (-0.05909493067851988, 0.06084993715202358)
Pearson value of  Min Vol  is  (-0.0660453130643115, 0.036124925653245085)
Pearson value of  Max Vol  is  (-0.09653387548359116, 0.002164397660592433)
Pearson value of  Min  is  (-0.06576512139327578, 0.03692348384649208)
Pearson value of  Max  is  (-0.06809058053773413, 0.030729304547231034)
Pearson value of  Window MA  is  (-0.06899316143752733, 0.02857736616001366)
Pearson value of  RSI 2W  is  (0.05137524170100003, 0.1032378350460256)
Pearson value of  RSI Window  is  (0.060603067889955564, 0.05454139040870371)
Pearson value of  Price Change  is  (0.0016287211177530639, 0.9588311028457459)
Pearson value of  Price Increase  is  (0.02144803761963549, 0.496600020701046)
Pearson value of  Avg Price Change  is  (0.04673514996090979, 0.1383332001734743)
Pearson value of  RSI Oversold  is  (0.03516679552330661, 0.26488642020644365)
Pearson value of  RSI Undersold  is  (0.02386835513775085, 0.4492968228625301)
'''
five_week_pearson = '''
Pearson value of  Avg Vol  is  (-0.05378461977677019, 0.08803107650169477)
Pearson value of  Min Vol  is  (-0.06613553118549591, 0.03587094937155756)
Pearson value of  Max Vol  is  (-0.06983091285414794, 0.026697259021720335)
Pearson value of  Min  is  (-0.06916302673070468, 0.028187203146641988)
Pearson value of  Max  is  (-0.06935203455292031, 0.027758467667381125)
Pearson value of  Window MA  is  (-0.07078875306559257, 0.024679251848746513)
Pearson value of  RSI 2W  is  (0.05137524170100003, 0.1032378350460256)
Pearson value of  RSI Window  is  (0.03696536701012476, 0.24120695445900686)
Pearson value of  Price Change  is  (0.0016287211177530639, 0.9588311028457459)
Pearson value of  Price Increase  is  (0.02144803761963549, 0.496600020701046)
Pearson value of  Avg Price Change  is  (0.0457926840827058, 0.14647151000229897)
Pearson value of  RSI Oversold  is  (0.03516679552330661, 0.26488642020644365)
Pearson value of  RSI Undersold  is  (0.02386835513775085, 0.4492968228625301)
'''